# Auricular shape analysis - curvature descriptors

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import trimesh
from PIL import Image
from scipy.ndimage import distance_transform_edt, binary_fill_holes

In [ ]:
sys.path.append("/home/vajicek/src/aurikular/shapeexplorer")

from projects.auricular.projection import computeHeightmap, regularSampling, getDistanceToEdge
from projects.auricular.curvaturedescriptors import CurvatureDescriptorsParams
from projects.auricular.curvaturedescriptors import CurvatureDescriptors
from projects.auricular.curvaturedescriptors import HistogramDescriptors
from projects.auricular.common import getSample

In [ ]:
input_data = os.path.expanduser("~/data/aurikularni_plocha_ply5/")
sample = list(getSample(input_data))
params = CurvatureDescriptorsParams(
    input_data=input_data,
    sampling_method='regular',
    dist=2.0,
    sampling_rate=0.5,
    sample_count=5000,
    output='../../../output_812')
cd = CurvatureDescriptors(params)

## Compute descriptors

In [ ]:
logging.basicConfig(level=logging.INFO)
cd.newAnalysis()
cd.computeDescriptors()

In [ ]:
data = cd.getData()

### Distance

In [ ]:
heightmap = computeHeightmap(trimesh.load_mesh(sample[10]['filename']), 1.0)
plt.imshow(heightmap)
plt.show()
plt.imshow(getDistanceToEdge(heightmap > np.min(heightmap)))

In [ ]:
bins = 15
normalize = False

hd = HistogramDescriptors(data, 0.5)
range2d = [[-10, 0],[0, 1 if normalize else 32]]
_ = plt.hist2d(*hd.getCurveDistFeatures(0, normalize), bins=bins, range=range2d)
plt.show()
_ = plt.hist2d(*hd.getCurveDistFeatures(811, normalize), bins=bins, range=range2d)

## Sample age distribution

In [ ]:
age=[float(specimen['age']) for specimen in sample]
plt.hist(age)
plt.show()
pd.DataFrame(age).describe()

## Reduced sample age distribution

In [ ]:
reduced_sample = list(getSample(os.path.expanduser("~/data/aurikularni_plocha_ply2/")))
reduced_sample_age = [float(specimen['age']) for specimen in reduced_sample]
plt.hist(reduced_sample_age)
plt.show()
pd.DataFrame(reduced_sample_age).describe()

## Data viewer

In [ ]:
trimesh.load(sample[3]['filename']).show()

In [ ]:
plt.imshow(Image.open(os.path.join('../../../output', sample[0]['basename'] + '_sample_map.png'), 'r'))

In [ ]:
plt.imshow(Image.open(os.path.join('../../../output', sample[307]['basename'] + '_sample_map.png'), 'r'))

## Load computed data

In [ ]:
data = cd.getData()
hist_descriptors = {
    0.5: HistogramDescriptors(data, 0.5),
    1.0: HistogramDescriptors(data, 1.0),
    2.0: HistogramDescriptors(data, 2.0)
}

In [ ]:
data[0]['dist'][2.0]['curvature'].mean()

In [ ]:
data[0]['dist'][2.0]['sampled_dne']

In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(data=[[
                    data1['name'],
                    data1['subset'],
                    data1['type'],
                    data1['side'],
                    data1['sex'],
                    float(data1['age']),
                    float(data1['dist'][2.0]['sampled_dne']),
                    np.log(float(data1['dist'][2.0]['sampled_dne']))] for data1 in data],
                  columns = ['name', 'subset', 'type', 'side', 'sex', 'age', 'meandne', 'log(meandne)'])

df.head()

In [ ]:
df['age'].hist();

In [ ]:
df.groupby(['subset'])['subset'].describe()[["count"]]

In [ ]:
cols = {a[0]: a[1] for a in zip(df['subset'].unique(), mcolors.BASE_COLORS.keys())}

fig, ax = plt.subplots(figsize=(15,15))
for subset, color in cols.items():
    df1 = df[df['subset']==subset]
    ax.scatter(df1['age'], df1['log(meandne)'], c=color, label=subset)
ax.legend()
df[['age','log(meandne)','name']].apply(lambda x: ax.text(*x), axis=1)
plt.xlabel("age")
plt.ylabel("log(meandne)");

In [ ]:
df.plot.scatter(x='age', y='meandne')